In [177]:
import math
import csv
import json

STAT_NAMES = [
    "caps", "garbage_time_caps", "hold", "ndps", "returns", "quick_returns", "nrts", "pups", 
    "keypops", "handoffs", "goodprevent", "resets", "badflaccids", "sparkedouts"
]

with open("../data/bulkmaps.json") as f:
    maps = json.load(f)

with open("smurfs.json") as f:
    smurfs = json.load(f)

def desmurf(player):
    player = player.replace("\\'", "'").replace("\\\\", "\\")
    if player in smurfs:
        return smurfs[player]
    return player

def row_to_dict(row):
    return {
        'match_id': int(row[0]),
        'map_name': maps[row[1]]['name'],
        'gamemode': maps[row[1]]['type'],
        'timestamp': int(row[2]),
        'duration': int(row[3]),
        'cap_diff': int(row[4]),
        'garbage_time_cap_diff': int(row[5]),
        'players': [
            {
                'name': desmurf(row[j + 6]),
                'stats': {
                    stat_name: int(row[j * len(STAT_NAMES) + 14 + i])
                    for i, stat_name in enumerate(STAT_NAMES)
                }
            }
            for j in range(8)
        ]
    }

with open("matchups_with_stats.csv") as f:
    reader = csv.reader(f)
    next(reader)
    matches = [row_to_dict(row) for row in reader]
    matches = [m for m in matches if m['gamemode'] == "ctf"]

elo = {}
variance = {}
pred_errors = []
winner_win_prob = []

In [178]:
STARTING_VARIANCE = 1.2
VARIANCE_TO_ADD_BACK = 0.002
BASE_VARIANCE = 4.5
RED_ADVANTAGE = 0.1
PLAYER_STAT_WEIGHTS_INITIAL = {
    "caps": 0.8,
    "garbage_time_caps": -0.3,
    "hold": 1.7 / 3600,
    "ndps": -0.5,
    "returns": 0.35,
    "quick_returns": 0.0,
    "nrts": 0.3,
    "pups": 0.3,
    "keypops": 0.0,
    "handoffs": 0.2,
    "goodprevent": 0.0,
    "resets": 0.0,
    "badflaccids": -0.0,
    "sparkedouts": 0.0
}
TEAM_STAT_WEIGHTS_INITIAL = {
    "caps": 0,
    "garbage_time_caps": -0.3,
    "hold": 0.8 / 3600,
    "ndps": -0.0,
    "returns": 0.04,
    "quick_returns": 0.0,
    "nrts": 0.0,
    "pups": 0.0,
    "keypops": -0.0,
    "handoffs": 0.0,
    "goodprevent": 0.0,
    "resets": 0.0,
    "badflaccids": -0.0,
    "sparkedouts": 0.0
}
PLAYER_STAT_WEIGHTS_CONVERGED = {
    "caps": 0.05,
    "garbage_time_caps": -0.0,
    "hold": 0.0 / 3600,
    "ndps": -0.0,
    "returns": 0.0,
    "quick_returns": 0.0,
    "nrts": 0.0,
    "pups": 0.0,
    "keypops": -0.0,
    "handoffs": 0.0,
    "goodprevent": 0.0,
    "resets": 0.0,
    "badflaccids": -0.0,
    "sparkedouts": 0.0
}
TEAM_STAT_WEIGHTS_CONVERGED = {
    "caps": 0,
    "garbage_time_caps": -0.0,
    "hold": 0.4 / 3600,
    "ndps": -0.0,
    "returns": 0.02,
    "quick_returns": 0.0,
    "nrts": 0.0,
    "pups": 0.0,
    "keypops": 0.0,
    "handoffs": 0.0,
    "goodprevent": 0.0,
    "resets": 0.0,
    "badflaccids": 0.0,
    "sparkedouts": 0.0
}
DIFF_MAPPING = [0, 0.2, 0.9, 2.1, 3.1, 4.2, -4.2, -3.1, -2.1, -0.9, -0.2]
NEW_PLAYER_ELO = -1.1
NEWNESS_THRESHOLD = 0.05
NEW_PLAYER_ELO_BOOST = 0.1
DIFF_WEIGHT_INITIAL = 0.65
DIFF_WEIGHT_CONVERGED = 0.6
GARBAGE_TIME_DISCOUNT = 0.35
RELATIVE_ELO_CORRECTION = 0.05

In [179]:
set([m['map_name'] for m in matches])

{'A Flaccid Type Map',
 'Apparition 2023',
 'Asida',
 'Audacity 2',
 'Basenji',
 'Catch-24',
 'Centenaria',
 'Combine',
 'Crawfish Boil',
 'GFY',
 'Haste',
 'Milano 2',
 'Moon Base 2024',
 'Nuke',
 'OTI Jardim',
 'OTI MERALD',
 'Oak',
 'Pilot 3',
 'Pine',
 'Pretzel 2',
 'Pukebox 2',
 'Sardonica',
 'Sugar Free Hill',
 'Tetanic',
 'Tetanic 3.1 4x4',
 'Thicket 2',
 'Transilio 2024',
 'Waterbed [MM23 West Winner]',
 'Willow 2'}

In [180]:
def get_elo(player):
    if player not in elo:
        elo[player] = NEW_PLAYER_ELO
    return elo[player]

def get_variance(player):
    if player not in variance:
        variance[player] = STARTING_VARIANCE
    return variance[player]

def normalize_elos():
    known_elos = [elo[p] for p in elo if variance[p] <= 0.2]
    avg_elo = sum(known_elos) / max(10, len(known_elos))
    for p in elo:
        elo[p] -= avg_elo
        if variance[p] < STARTING_VARIANCE / 2:
            variance[p] = min(STARTING_VARIANCE / 2, variance[p] + VARIANCE_TO_ADD_BACK)

In [181]:
def judge_stats_initial(players):
    scores = [
        sum([
            p['stats'][stat_name] * PLAYER_STAT_WEIGHTS_INITIAL[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players
    ]
    team_score_diff = sum([
        sum([
            p['stats'][stat_name] * TEAM_STAT_WEIGHTS_INITIAL[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players[:4]
    ]) - sum([
        sum([
            p['stats'][stat_name] * TEAM_STAT_WEIGHTS_INITIAL[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players[4:]
    ])
    red_avg_score = sum(scores[:4]) / 4
    blue_avg_score = sum(scores[4:]) / 4
    red_scores = [s - red_avg_score + team_score_diff for s in scores[:4]]
    blue_scores = [s - blue_avg_score - team_score_diff for s in scores[4:]]
    return red_scores, blue_scores

def judge_stats_converged(players):
    scores = [
        sum([
            p['stats'][stat_name] * PLAYER_STAT_WEIGHTS_CONVERGED[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players
    ]
    team_score_diff = sum([
        sum([
            p['stats'][stat_name] * TEAM_STAT_WEIGHTS_CONVERGED[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players[:4]
    ]) - sum([
        sum([
            p['stats'][stat_name] * TEAM_STAT_WEIGHTS_CONVERGED[stat_name]
            for stat_name in STAT_NAMES
        ])
        for p in players[4:]
    ])
    red_avg_score = sum(scores[:4]) / 4
    blue_avg_score = sum(scores[4:]) / 4
    red_scores = [s - red_avg_score + team_score_diff for s in scores[:4]]
    blue_scores = [s - blue_avg_score - team_score_diff for s in scores[4:]]
    return red_scores, blue_scores

def update_rating(player, error, stat_score_initial, stat_score_converged, total_variance, elo_vs_game_avg):
    share_of_variance = variance[player] / total_variance
    newness = variance[player] / STARTING_VARIANCE
    error_weight = DIFF_WEIGHT_INITIAL * newness + DIFF_WEIGHT_CONVERGED * (1 - newness)
    stat_score = stat_score_initial * newness + stat_score_converged * (1 - newness)
    update = error_weight * error + stat_score
    update -= elo_vs_game_avg * RELATIVE_ELO_CORRECTION
    
    elo[player] += update * share_of_variance
    variance[player] *= 1 - share_of_variance
    if variance[player] > NEWNESS_THRESHOLD:
        elo[player] += NEW_PLAYER_ELO_BOOST * share_of_variance

def update_ratings(match):
    expected_score = RED_ADVANTAGE
    total_variance = BASE_VARIANCE
    
    for player in match['players'][:4]:
        expected_score += get_elo(player['name'])
        total_variance += get_variance(player['name'])
    for player in match['players'][4:]:
        expected_score -= get_elo(player['name'])
        total_variance += get_variance(player['name'])
    
    all_player_names = [p['name'] for p in match['players']]
    all_player_elos = [get_elo(name) for name in all_player_names]
    game_avg_elo = sum(all_player_elos) / len(all_player_elos)

    winner_win_prob.append(1 / (1 + math.exp(-expected_score / 2 if match['cap_diff'] > 0 else expected_score / 2)))
    pred_errors.append(match['cap_diff'] - min(max(expected_score, -5), 5))
    error = DIFF_MAPPING[match['cap_diff']] -\
        match['garbage_time_cap_diff'] * GARBAGE_TIME_DISCOUNT -\
        min(max(expected_score, DIFF_MAPPING[-5]), DIFF_MAPPING[5])
    
    red_stats_initial, blue_stats_initial = judge_stats_initial(match['players'])
    red_stats_converged, blue_stats_converged = judge_stats_converged(match['players'])
    for player, stat_score_initial, stat_score_converged in zip(match['players'][:4], red_stats_initial, red_stats_converged):
        elo_vs_game_avg = get_elo(player['name']) - game_avg_elo
        update_rating(player['name'], error, stat_score_initial, stat_score_converged, total_variance, elo_vs_game_avg)
    for player, stat_score_initial, stat_score_converged in zip(match['players'][4:], blue_stats_initial, blue_stats_converged):
        elo_vs_game_avg = get_elo(player['name']) - game_avg_elo
        update_rating(player['name'], -error, stat_score_initial, stat_score_converged, total_variance, elo_vs_game_avg)

In [182]:
elo = {}
variance = {}
pred_errors = []
winner_win_prob = []
day = 0

for match in matches:
    new_day = round(match['timestamp'] / 86400)
    if new_day != day:
        normalize_elos()
    day = new_day
    update_ratings(match)

print(f"MAE: {sum([abs(e) for e in pred_errors]) / len(pred_errors):.4f}    MSE: {sum([e ** 2 for e in pred_errors]) / len(pred_errors):.4f}")
print(f"COR: {sum([(1 if x > 0.5 else (0.5 if x == 0.5 else 0)) for x in winner_win_prob]) / len(winner_win_prob):.2%}    WWP: {sum(winner_win_prob) / len(winner_win_prob):.2%}")
print(f"BRI: {1 - (sum([(1 - x) ** 2 for x in winner_win_prob]) / len(winner_win_prob)) ** 0.5:.2%}    LOG: {2 ** (sum([math.log2(x) for x in winner_win_prob]) / len(winner_win_prob)):.2%}")

MAE: 2.3867    MSE: 8.3074
COR: 59.60%    WWP: 52.63%
BRI: 51.49%    LOG: 51.55%


In [183]:
print(f"MAE: {sum([abs(e) for e in pred_errors[20000:]]) / len(pred_errors[20000:]):.4f}    MSE: {sum([e ** 2 for e in pred_errors[20000:]]) / len(pred_errors[20000:]):.4f}")
print(f"COR: {sum([(1 if x > 0.5 else (0.5 if x == 0.5 else 0)) for x in winner_win_prob[20000:]]) / len(winner_win_prob[20000:]):.2%}    WWP: {sum(winner_win_prob[-20000:]) / len(winner_win_prob[-20000:]):.2%}")
print(f"BRI: {1 - (sum([(1 - x) ** 2 for x in winner_win_prob[20000:]]) / len(winner_win_prob[20000:])) ** 0.5:.2%}    LOG: {2 ** (sum([math.log2(x) for x in winner_win_prob[-20000:]]) / len(winner_win_prob[-20000:])):.2%}")

MAE: 2.3817    MSE: 8.2719
COR: 58.84%    WWP: 52.23%
BRI: 51.23%    LOG: 51.26%


In [184]:
leaderboard = sorted([
    p for p in elo if variance[p] <= 0.21
], key=lambda x: elo[x], reverse=True)
for i, player in enumerate(leaderboard):
    print(f"{i + 1:>3}.  {player:<12}  {elo[player]:5.2f} ± {1.96 * variance[player] ** 0.5:.2f}")

  1.  SluffAndRuff   2.81 ± 0.63
  2.  okthen         2.73 ± 0.74
  3.  DT             2.65 ± 0.67
  4.  may_hem        2.58 ± 0.35
  5.  Junoon         2.57 ± 0.32
  6.  Alphachurro    2.35 ± 0.46
  7.  Xx360NoSwagx   2.33 ± 0.73
  8.  phreak         2.25 ± 0.52
  9.  Ritual         2.15 ± 0.57
 10.  Shikari        2.14 ± 0.59
 11.  mex            2.12 ± 0.35
 12.  bright         2.11 ± 0.48
 13.  jig            2.11 ± 0.48
 14.  Suchit         2.05 ± 0.34
 15.  BALLDON'TLIE   2.05 ± 0.49
 16.  danp           2.05 ± 0.37
 17.  meowza         2.03 ± 0.53
 18.  realtea        1.97 ± 0.46
 19.  OuchMyBalls    1.97 ± 0.36
 20.  fender         1.90 ± 0.50
 21.  Messi          1.88 ± 0.39
 22.  Sif            1.86 ± 0.34
 23.  ft             1.78 ± 0.46
 24.  1deag          1.75 ± 0.47
 25.  Ty             1.75 ± 0.83
 26.  rina           1.71 ± 0.64
 27.  Doris          1.70 ± 0.65
 28.  Enervate       1.69 ± 0.41
 29.  BallsToYou.    1.66 ± 0.46
 30.  Maelstrom      1.65 ± 0.38
 31.  bbb 

In [185]:
for i, p in enumerate(leaderboard):
    print("\t".join([str(i + 1), p, f"{elo[p]:.2f}", f"{1.96 * variance[p] ** 0.5:.2f}"]))

1	SluffAndRuff	2.81	0.63
2	okthen	2.73	0.74
3	DT	2.65	0.67
4	may_hem	2.58	0.35
5	Junoon	2.57	0.32
6	Alphachurro	2.35	0.46
7	Xx360NoSwagx	2.33	0.73
8	phreak	2.25	0.52
9	Ritual	2.15	0.57
10	Shikari	2.14	0.59
11	mex	2.12	0.35
12	bright	2.11	0.48
13	jig	2.11	0.48
14	Suchit	2.05	0.34
15	BALLDON'TLIE	2.05	0.49
16	danp	2.05	0.37
17	meowza	2.03	0.53
18	realtea	1.97	0.46
19	OuchMyBalls	1.97	0.36
20	fender	1.90	0.50
21	Messi	1.88	0.39
22	Sif	1.86	0.34
23	ft	1.78	0.46
24	1deag	1.75	0.47
25	Ty	1.75	0.83
26	rina	1.71	0.64
27	Doris	1.70	0.65
28	Enervate	1.69	0.41
29	BallsToYou.	1.66	0.46
30	Maelstrom	1.65	0.38
31	bbb	1.64	0.44
32	grunt	1.62	0.58
33	tits	1.61	0.70
34	Mr awesome:)	1.60	0.77
35	SleepyBoi	1.59	0.64
36	kool aid	1.59	0.86
37	Crippy	1.56	0.67
38	Russ	1.54	0.49
39	jazzz	1.53	0.70
40	ASAP	1.53	0.74
41	kcx	1.52	0.82
42	joy.	1.52	0.43
43	Button	1.49	0.48
44	SakuUta	1.48	0.55
45	BallSaget	1.47	0.43
46	Vader	1.47	0.87
47	karma	1.46	0.42
48	eee	1.45	0.74
49	Homie	1.40	0.59
50	tears	1.39	0.55
51	d

In [186]:
import random

ranks = []
for _ in range(10000):
    simulated_elos = {p: elo[p] + variance[p] ** 0.5 * random.gauss(0, 1) for p in elo if variance[p] <= 0.3}
    sorted_players = sorted(simulated_elos.items(), key=lambda x: x[1], reverse=True)
    ranks.append({player: rank + 1 for rank, (player, _) in enumerate(sorted_players)})

for i, player in enumerate(leaderboard[:]):
    rank_ci = sorted([r[player] for r in ranks])[500:-500]
    print(f"{i + 1:>3}.  {player:<12}  {elo[player]:5.2f} ± {1.96 * variance[player] ** 0.5:.2f}  ({min(rank_ci)}-{max(rank_ci)})")

  1.  SluffAndRuff   2.81 ± 0.63  (1-10)
  2.  okthen         2.73 ± 0.74  (1-15)
  3.  DT             2.65 ± 0.67  (1-16)
  4.  may_hem        2.58 ± 0.35  (2-10)
  5.  Junoon         2.57 ± 0.32  (2-10)
  6.  Alphachurro    2.35 ± 0.46  (3-21)
  7.  Xx360NoSwagx   2.33 ± 0.73  (2-33)
  8.  phreak         2.25 ± 0.52  (4-28)
  9.  Ritual         2.15 ± 0.57  (4-37)
 10.  Shikari        2.14 ± 0.59  (4-38)
 11.  mex            2.12 ± 0.35  (7-28)
 12.  bright         2.11 ± 0.48  (6-34)
 13.  jig            2.11 ± 0.48  (6-35)
 14.  Suchit         2.05 ± 0.34  (9-31)
 15.  BALLDON'TLIE   2.05 ± 0.49  (7-38)
 16.  danp           2.05 ± 0.37  (8-32)
 17.  meowza         2.03 ± 0.53  (6-42)
 18.  realtea        1.97 ± 0.46  (8-42)
 19.  OuchMyBalls    1.97 ± 0.36  (10-37)
 20.  fender         1.90 ± 0.50  (9-49)
 21.  Messi          1.88 ± 0.39  (12-44)
 22.  Sif            1.86 ± 0.34  (14-43)
 23.  ft             1.78 ± 0.46  (13-56)
 24.  1deag          1.75 ± 0.47  (14-59)
 25.  Ty   